In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import pickle
import numpy as np
from llm_utils import cer_with_gpt2_decoder

In [3]:
model_name = "facebook/opt-6.7b"

# Load tokenizer
llm_tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model in 8-bit with automatic device placement
llm = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto"
)

# Example: Generate from a prompt
inputs = llm_tokenizer("The future of AI is", return_tensors="pt").to(llm.device)
outputs = llm.generate(**inputs, max_new_tokens=50)
print(llm_tokenizer.decode(outputs[0], skip_special_tokens=True))

Skipping import of cpp extensions due to incompatible torch version 2.10.0+cu128 for torchao version 0.15.0             Please see https://github.com/pytorch/ao/issues/2919 for more info
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The future of AI is in the hands of the people

The future of AI is in the hands of the people

The future of AI is in the hands of the people

The future of AI is in the hands of the people

The future of


In [4]:
acoustic_scale = 0.5
llm_weight = 0.5
seed_list = [0]

In [5]:
nbest_path = "/home/ebrahim/data2/brain2text/b2t_24/model_outputs/bidirectional_gru/bidirectional_gru_seed_0_nbest_5gram.pkl"
model_outputs_path = "/home/ebrahim/data2/brain2text/b2t_24/model_outputs/bidirectional_gru/bidirectional_gru_seed_0_model_outputs.pkl"

with open(model_outputs_path, mode = 'rb') as f:
    model_outputs = pickle.load(f)
    
with open(nbest_path, mode = 'rb') as f:
    nbest = pickle.load(f)

for i in range(len(model_outputs['transcriptions'])):
    new_trans = [ord(c) for c in model_outputs['transcriptions'][i]] + [0]
    model_outputs['transcriptions'][i] = np.array(new_trans)
    

# Rescore nbest outputs with LLM
llm_out = cer_with_gpt2_decoder(
    llm,
    llm_tokenizer,
    nbest[:],
    acoustic_scale,
    model_outputs,
    outputType="speech_sil",
    returnCI=True,
    lengthPenalty=0,
    alpha=llm_weight,
)


with open("/home/ebrahim/data2/brain2text/b2t_24/model_outputs/bidirectional_gru/bidirectional_gru_seed_0_llm_outs.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(llm_out['decoded_transcripts'])+ "\n")   # one line per LLM output

  0%|          | 0/880 [00:00<?, ?it/s]

In [ ]:
print("HI")

In [6]:
llm_out["wer"]
    

(np.float64(0.15439170758319695),
 np.float64(0.14027706574962823),
 np.float64(0.16899384824954927))